<a href="https://colab.research.google.com/github/zliobaite/TBIteaching2024/blob/main/Practical2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Practical 2: predictive models to analyze collecting strategies**

In this exercise you will use a dataset recording collecting decisions in fieldwork. You will make predictive models to predict the probability of collection from the taxonomic information, size and preservation of a fossil. Two types of models will be built -- a logistic regression (global model) and a decision tree (local model).

Compare the training, hold out and cross validation accuracies.

**Step 1:** load the dataset from a csv

In [ ]:
# deleting possible previous uploads
import os
os.remove("TMP_census_2022_release.csv")

from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import io

df = pd.read_csv(io.StringIO(uploaded['TMP_census_2022_release.csv'].decode('utf-8')),delimiter=',')
df.head()

**Step 2:** make a logistic regression model

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Replace missing values "?" with NaN
df.replace('?', pd.NA, inplace=True)

# Selecting input columns and the target variable
X = df[['Order', 'Bigpart', 'Larger', 'Preservation', 'Completeness']]
y = df['Collected']

# Convert categorical variables using pd.get_dummies
X_encoded = pd.get_dummies(X, columns=['Order', 'Bigpart'], drop_first=False)

# Convert "Larger" to numeric (1 for 'yes', 0 for 'no' and missing values as 0.5)
mapping = {'yes': 1, 'no': 0}
X_encoded['Larger'] = X['Larger'].map(mapping).fillna(0.5)


# Impute missing values in numerical variables
X_numerical = X[['Preservation', 'Completeness']].replace({pd.NA: np.nan})
imputer = SimpleImputer(strategy='mean')
X_encoded[['Preservation', 'Completeness']] = imputer.fit_transform(X_numerical)

# Standardize numerical variables
scaler = StandardScaler()
X_encoded[['Preservation', 'Completeness']] = scaler.fit_transform(X_encoded[['Preservation', 'Completeness']])


In [ ]:
#fitting the model on the full dataset
model = LogisticRegression()
model.fit(X_encoded, y)

#model
y_pred_full = model.predict(X_encoded)

# accuracy scores
accuracy = accuracy_score(y, y_pred_full)
print(f'Accuracy: {accuracy:.2f}')

print(classification_report(y, y_pred_full))

In [ ]:
# visualize regression coefficients as a histogram

import matplotlib.pyplot as plt

# Print the coefficients including intercept
intercept = model.intercept_[0]
coefficients = model.coef_[0]

print(f"Intercept: {intercept:.2f}")
for feature, coef in zip(X_encoded.columns, coefficients):
    print(f"{feature}: {coef:.2f}")

# Plotting coefficients as a histogram
plt.figure(figsize=(8, 6))
plt.bar(range(len(coefficients)), coefficients)
plt.xticks(range(len(coefficients)), X_encoded.columns, rotation=90)
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.5)  # Add a horizontal line at y=0
plt.title('Logistic Regression Coefficients')
plt.xlabel('Features')
plt.ylabel('Coefficient Value')
plt.tight_layout()
plt.show()

In [ ]:
# testing on a hold out testing set

# train test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)


model.fit(X_train, y_train)
# model predictions on the test set
y_pred = model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Classification report
print(classification_report(y_test, y_pred))



In [ ]:
# test via cross validation without redoing preprocessing

from sklearn.model_selection import cross_val_score, StratifiedKFold

# Create and fit the model
model = LogisticRegression()
model.fit(X_encoded, y)

# Define cross-validation strategy
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation on training data
scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')

# Round the scores to two decimal places
rounded_scores = np.round(scores, 2)
mean_accuracy = np.round(scores.mean(), 2)

# Print rounded accuracy scores for each fold
print("Cross-validation accuracy per fold:", rounded_scores)
print("Mean accuracy:", mean_accuracy)


**Step 3:** decision tree

In [ ]:
# Create decision tree classifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

dt_classifier = DecisionTreeClassifier(max_depth=5, random_state=42)

# Fit the classifier on the training data
dt_classifier.fit(X_encoded, y)

# Predict on the test set
y_pred = dt_classifier.predict(X_encoded)

# Evaluate accuracy on the test set
accuracy = accuracy_score(y, y_pred)
print("Decision tree training accuracy:", accuracy)


In [ ]:
# visualize

# Visualize the decision tree
#plt.figure(figsize=(12, 8))
#plot_tree(dt_classifier, feature_names=X_encoded.columns, class_names=['no', 'yes'], filled=True, fontsize=10)
#plt.title("Decision Tree Visualization")
#plt.show()

def plot_decision_tree(clf, feature_names, class_names):
    plt.figure(figsize=(16, 10))
    plot_tree(clf, feature_names=feature_names, class_names=class_names, filled=True, fontsize=10, impurity=False, rounded=True)
    plt.show()

# Visualize the simplified decision tree
plot_decision_tree(dt_classifier, X_encoded.columns, dt_classifier.classes_)


In [ ]:
# testing on a hold out testing set


dt_classifier = DecisionTreeClassifier(max_depth=5, random_state=42)
dt_classifier.fit(X_train, y_train)

y_pred = dt_classifier.predict(X_test)
#print(y_pred)

accuracy = accuracy_score(y_test, y_pred)
print("Decision tree hold out test accuracy:", accuracy)



In [ ]:
# Define cross-validation strategy
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation on training data
cv_scores = cross_val_score(dt_classifier, X_encoded, y, cv=cv, scoring='accuracy')

# Round the scores to two decimal places
rounded_scores = np.round(cv_scores, 2)
mean_accuracy = np.round(cv_scores.mean(), 2)

# Print rounded accuracy scores for each fold
print("Cross-validation accuracy per fold:", rounded_scores)
print("Mean accuracy:", mean_accuracy)
